In [1]:
import math

## Rychlost zvuku ve vzduchu v závislosti na teplotě

Jednoduchá funkce pro výpočet rychlosti zvuku v závislosti na teplotě ve °C

Dle vztahu:
$$
v(T) = 331.57 + 0.607 \cdot T
$$
Kde $\{T\}=°C$

In [2]:
def sound_wave_velocity( temperature : float ) -> float:
    return float(331.57 + 0.607 * temperature)

## Vlnová dělka

Výpočet vlnové délky v závislosti na frekvenci vlnění a jeho rychlosti.

Dle vztahu:
$$
\lambda(v,f) = \frac{v}{f} 
$$

Vstupní parametry:
* `freq` - frekvence
* `velocity` - rychlost

In [3]:
def sound_wavelength( freq : float, velocity : float ) -> float:
    return float( velocity / freq )

## Endfire/inline gradient spacing

In [4]:
def sub_spacing( freq : float, temperature : float ) -> float:
    velocity = sound_wave_velocity(temperature)
    wavelength = sound_wavelength(freq, velocity)
    return float(wavelength / 4)

In [24]:
print(sub_spacing(60, 20))

1.4321249999999999


## FAR

[https://www.sounddesignlive.com/one-simple-tool-find-right-size-speaker-space/](https://www.sounddesignlive.com/one-simple-tool-find-right-size-speaker-space/)

In [6]:
def FAR_coefficient( depth : float, width : float ) -> float:
    return float( depth / width )

### Převod FAR na vyzařovací úhel

In [7]:
def angle_from_FAR( far_coef : float ) -> float:
    angle_in_rads = 2 * math.asin( 1 / far_coef )
    return float( angle_in_rads * (360 / (2 * math.pi)))

In [26]:
depth = 30
width = 23
FAR = FAR_coefficient( depth=depth, width=width )
angle = angle_from_FAR( FAR )

print("FAR: {0} ; ANGLE: {1}".format(FAR, angle))

FAR: 1.3043478260869565 ; ANGLE: 100.11098962037022


### Endfire coords

#### Objekt reperezentující bodový zdroj (basovku)

In [9]:
from dataclasses import dataclass

In [10]:
@dataclass
class Pointsource:
    x : float
    y : float
    z : float
    heading_angle : float
    inclination_angle : float
    delay : float

#### Generování souřadnic pro endfire

In [11]:
def generate_endfire_coords( frequency : float, temperature : float, number_of_cabs : int, shift : float ) -> list:
    cabinet_list = list()
    
    velocity = sound_wave_velocity( temperature )
    spacing = sub_spacing( frequency, temperature )
    
    for cabinet_num in range( number_of_cabs ):
        x_coord = shift + cabinet_num * spacing
        delay = cabinet_num * ( spacing / velocity ) * 1000
        
        point_source = Pointsource(
            x=x_coord,
            y=0,
            z=0,
            heading_angle=0,
            inclination_angle=0,
            delay=delay
        )
        
        cabinet_list.append( point_source )
    
    return cabinet_list

In [18]:
cabinet_list = generate_endfire_coords(
    frequency=50,
    temperature=20,
    number_of_cabs=4,
    shift=1
)

In [19]:
def print_cabinets( cabinet_list : list ) -> None:
    for index, cabinet in enumerate(cabinet_list):
        print("Cabinet: {0} ; x={1} ; y={2} ; z={3} ; heading={4} ; delay={5}".format(index + 1, cabinet.x, cabinet.y, cabinet.z, cabinet.heading_angle, cabinet.delay))

In [20]:
print_cabinets(cabinet_list)

Cabinet: 1 ; x=1.0 ; y=0 ; z=0 ; heading=0 ; delay=0.0
Cabinet: 2 ; x=2.7185499999999996 ; y=0 ; z=0 ; heading=0 ; delay=5.0
Cabinet: 3 ; x=4.437099999999999 ; y=0 ; z=0 ; heading=0 ; delay=10.0
Cabinet: 4 ; x=6.15565 ; y=0 ; z=0 ; heading=0 ; delay=15.0


#### Rotace endfire

$$ \sin(\varphi) = \frac{y}{spacing} \rightarrow y = \sin(\varphi) \cdot spacing  \\ \cos(\varphi) = \frac{x}{spacing} \rightarrow x = \cos(\varphi) \cdot spacing $$

In [21]:
def rotate_endfire( cabinet_list : list, angle : float ) -> list:
    
    angle_in_rads = ((2 * math.pi) / 360) * angle
    print(angle_in_rads)
    for cabinet in cabinet_list:
        spacing = cabinet.x
        cabinet.y = math.sin( angle_in_rads ) * spacing
        cabinet.x = math.cos( angle_in_rads ) * spacing
        cabinet.heading_angle = angle
    
    return cabinet_list

In [22]:
rotated_cabinets = rotate_endfire(cabinet_list, 30)

0.5235987755982988


In [23]:
print_cabinets(rotated_cabinets)

Cabinet: 1 ; x=0.8660254037844387 ; y=0.49999999999999994 ; z=0 ; heading=30 ; delay=0.0
Cabinet: 2 ; x=2.3543333614581856 ; y=1.3592749999999996 ; z=0 ; heading=30 ; delay=5.0
Cabinet: 3 ; x=3.8426413191319324 ; y=2.218549999999999 ; z=0 ; heading=30 ; delay=10.0
Cabinet: 4 ; x=5.33094927680568 ; y=3.0778249999999994 ; z=0 ; heading=30 ; delay=15.0


#### Vzdálenost 2 bodů

* Mějme 2 body:
    * $X[x_1,x_2,x_3]$
    * $Y[y_1,y_2,y_3]$
* Vzdálenost těchto bodů $|XY|$:
$$
|XY| = \sqrt{(y_1 - x_1)^2 + (y_2 - x_2)^2 + (y_3 - x_3)^2}
$$

In [50]:
def distance_of_two_points( x1 : float, x2 : float, x3 : float, y1 : float, y2 : float, y3 : float ) -> float:
    distance = math.sqrt((y1 - x1)**2 + (y2 - x2)**2 + (y3 - x3)**2)
    return distance

In [51]:
distance_of_two_points(2,2,0,0,0,0)

2.8284271247461903

#### Výpočet dráhových a časových rozdílů pro určitý bod

In [52]:
def adjust_delays( cabinets : list, point_x, point_y, point_z, temp : float ) -> list:
    return_cabs = list()
    
    first_cabinet = cabinets[0]
    
    distance_to_first = distance_of_two_points(
        x1=first_cabinet.x,
        x2=first_cabinet.y,
        x3=first_cabinet.z,
        y1=point_x,
        y2=point_y,
        y3=point_z
    )
    
    return_cabs.append(first_cabinet)
    
    print(distance_to_first)
    
    for index, cabinet in enumerate(cabinets[1:]):
        distance = distance_of_two_points(
            x1 = cabinet.x,
            x2 = cabinet.y,
            x3 = cabinet.z,
            y1 = point_x,
            y2 = point_y,
            y3 = point_z
        )
        
        distance_dif = distance_to_first - distance
        
        cabinet.delay = (distance_dif / sound_wave_velocity( temp )) * 1000
        return_cabs.append(cabinet)
        
        print("Dst={0} ; Diff={1}".format(distance, distance_dif))
    
    return return_cabs


In [53]:
adjusted_cabs = adjust_delays(rotated_cabinets, 10, 0, 0, 20)

9.539392014169456
Dst=8.955725213655228 ; Diff=0.5836668005142283
Dst=8.67852847031108 ; Diff=0.8608635438583754
Dst=8.737020483122377 ; Diff=0.8023715310470791


In [54]:
print_cabinets(adjusted_cabs)

Cabinet: 1 ; x=0.5000000000000001 ; y=0.8660254037844386 ; z=0 ; heading=60 ; delay=0.0
Cabinet: 2 ; x=1.359275 ; y=2.354333361458185 ; z=0 ; heading=60 ; delay=1.6981373847552539
Cabinet: 3 ; x=2.21855 ; y=3.842641319131932 ; z=0 ; heading=60 ; delay=2.5046217563014617
Cabinet: 4 ; x=3.0778250000000007 ; y=5.330949276805679 ; z=0 ; heading=60 ; delay=2.3344433710019468


### Úbytek akustického tlaku na vzdálenosti

In [6]:
def sound_pressure_level_change_from_distance_offset( first_distance : float, second_distance : float ) -> float:
    return 20*math.log10( first_distance / second_distance )
    

In [8]:
first_distance = 1.0
second_distance = 2.0
spl_diff = sound_pressure_level_change_from_distance_offset( first_distance, second_distance )
print("Attenuation: {0}".format(spl_diff))

Attenuation: -6.020599913279624
